## Relayed HOPR tokens by the ct-app

In [1]:
from dotenv import load_dotenv

from core.components.parameters import Parameters
from core.components.utils import Utils
from core.model.subgraph_entry import SubgraphEntry
from core.model.topology_entry import TopologyEntry
from core.model.economic_model import EconomicModel
from core.model.peer import Peer
from core.node import Node


In [2]:
# Load environment variables
load_dotenv("relayed_token.env")

params = Parameters()("SUBGRAPH_", "GCP_", "ECONOMIC_MODEL_")
nodes = Node.fromAddressListAndKey(*Utils.nodesAddresses("NODE_ADDRESS_", "NODE_KEY"))

api = nodes[-1].api
network_nodes = nodes[:-1]


In [3]:
async def get_subgraph_data():
    data = {
        "query": params.subgraph.query,
        "variables": {"first": params.subgraph.pagination_size, "skip": 0},
    }

    safes = []
    while True:
        _, response = await Utils.httpPOST(
            params.subgraph.url, data
        )

        if "data" not in response:
            break

        safes.extend(response["data"]["safes"])

        if len(response["data"]["safes"]) >= params.subgraph.pagination_size:
            data["variables"]["skip"] += params.subgraph.pagination_size
        else:
            break

    results = list[SubgraphEntry]()
    for safe in safes:
        results.extend(
            [
                SubgraphEntry.fromSubgraphResult(node)
                for node in safe["registeredNodesInNetworkRegistry"]
            ]
        )


    return results

async def get_topology_data():
    channels = await api.all_channels(False)

    results = await Utils.aggregatePeerBalanceInChannels(channels.all)
    return [TopologyEntry.fromDict(*arg) for arg in results.items()]

async def get_node_data():
    results = set[Peer]()

    for node in network_nodes:
        await node._retrieve_address()
        node_result = await node.api.peers(params=["peer_id", "peer_address"], quality=0.5)
        
        peers = {Peer(item["peer_id"], item["peer_address"]) for item in node_result}
        results.update(peers)

    return results

In [4]:
topology = await get_topology_data()
print(f"Topology size: {len(topology)}")

subgraph = await get_subgraph_data()
print(f"Subgraph size: {len(subgraph)}")

peers = await get_node_data()
print(f"Number of peers: {len(peers)}")

Topology size: 387
Subgraph size: 544
Number of peers: 359


In [5]:
eligible = Utils.mergeTopologyPeersSubgraph(topology, peers, subgraph)
Utils.allowManyNodePerSafe(eligible)
model = EconomicModel.fromGCPFile(
    params.gcp.bucket, params.economic_model.filename
)
for peer in eligible:
    peer.economic_model = model
    
Utils.rewardProbability(eligible)

print(f"Eligible peers: {len(eligible)}")

Eligible peers: 281


### Funds of Netwatchers 

In [6]:
node_funds = []

for node in network_nodes:
    peer = [peer for peer in eligible if peer.address == node.address][0]
    node_funds.append(peer.total_balance)

print(node_funds)

[35351.29999999998, 35378.20000000001, 35854.69999999999, 36506.19999999998]


### Relayed Token Calculation 

In [7]:
ct_funding_received = 200_000 # Funding transactions that the ct-app receives (should be automated by getting that information directly from the chain)
distributed_rewards = ct_funding_received - sum(node_funds)
print(f"{distributed_rewards:_.2f}")

56_909.60


In [10]:
airdropped = 6*25_000 # Get this number from Andrius
total_rewards = airdropped + distributed_rewards
print(f"{total_rewards:_.2f}")

206_909.60
